## Producer 2

**Description**: 
Gets the rating csv's from the datasink folder and sends them to the specified framework kafka or rabbitmq.

In [ ]:
import json
import time
from helper_file import Producer
import pandas as pd
import os

topic = "top_ratings"
path = "src/datasink/"
hz = 1/10

if not os.path.exists(path+"archive/"):
    os.makedirs(path+"archive/")

# producer = Producer(framework='kafka', host_name="broker1", port=9093)
producer_2 = Producer(framework='rabbitmq', host_name="rabbitmq1", port=5672)

while True:
    for file in os.listdir(path):
        if file.endswith(".csv"):
            df = pd.read_csv(path+file)
            df = df.head(5)
            message = df.to_json()
            producer_2.produce(topic, message)
            print(f"File: {file} published")
            os.replace(f"{path}{file}", f"{path}archive/{file}")
    time.sleep(1/hz)